In [ ]:
!pip install torch transformers datasets pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import torch
from datasets import load_dataset
import os

In [ ]:
dataset= load_dataset("community-datasets/offenseval_dravidian", "malayalam")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset['train'])
df = df.dropna(subset=['label'])

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'])

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
MAX_LEN = 128
BATCH_SIZE = 16

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Number of devices:", torch.cuda.device_count())
    # Set device to the first available GPU
    device = torch.device("cuda:0")
    print("Current device:", device)
else:
    print("CUDA is not available.")
device = torch.device("cuda")
train_dataset = TextClassificationDataset(
    texts=train_df['text'].to_numpy(),
    labels=train_df['label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

val_dataset = TextClassificationDataset(
    texts=val_df['text'].to_numpy(),
    labels=val_df['label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)


CUDA is available. Number of devices: 1
Current device: cuda:0


In [ ]:
import torch

# Check for available devices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)

# Move model to device
model.to(device)

Using device: cuda


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
10,1.659100
20,1.639800
30,1.585900
40,1.517300
50,1.379500
60,1.216500
70,0.902300
80,0.544500
90,0.565000
100,0.487500


TrainOutput(global_step=2403, training_loss=0.2632554149533429, metrics={'train_runtime': 574.8526, 'train_samples_per_second': 66.841, 'train_steps_per_second': 4.18, 'total_flos': 1272572603117568.0, 'train_loss': 0.2632554149533429, 'epoch': 3.0})

In [ ]:
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")

Evaluation results: {'eval_loss': 0.2511845529079437, 'eval_runtime': 12.3731, 'eval_samples_per_second': 258.786, 'eval_steps_per_second': 16.245, 'epoch': 3.0}


In [ ]:
def predict(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        _, prediction = torch.max(outputs.logits, dim=1)
    return prediction.item()

In [ ]:
new_text = "Ee asurante varavin katta waiting........MAMMOOKKA.. uyir"
label = predict(new_text)
print(f"Predicted label: {label}")


Predicted label: 0


In [ ]:
new_text = "Itinum dislike adikkan aalundallo...nallatine angeegarikkan padikk... duranthangal..."
label = predict(new_text)
print(f"Predicted label: {label}")

Predicted label: 1


In [ ]:
import torch

# Save the model's state_dict to a .pt file
torch.save(model.state_dict(), 'malayamodel_state_dict.pt')

# Save the entire model (including the architecture)
torch.save(model, 'malayamodel.pt')

# Save the tokenizer
tokenizer.save_pretrained('malayamodel_tokenizer')

('malayamodel_tokenizer/tokenizer_config.json',
 'malayamodel_tokenizer/special_tokens_map.json',
 'malayamodel_tokenizer/vocab.txt',
 'malayamodel_tokenizer/added_tokens.json')

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the state_dict
#model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)
#model.load_state_dict(torch.load('/content/drive/MyDrive/models/tammodel_state_dict.pt'))

# Alternatively, load the entire model
model = torch.load('/content/drive/MyDrive/models/malayamodel.pt')

# Set the model to evaluation mode
model.eval()

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('/content/drive/MyDrive/models/malayamodel_tokenizer')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

import torch
from transformers import DistilBertTokenizer

# Function to make predictions
def predict(texts, model, tokenizer, device, max_len=128):
    model.to(device)
    model.eval()

    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)
    return predictions.cpu()  # Move predictions back to CPU

# Example usage
texts = [
    "Ee asurante varavin katta waiting........MAMMOOKKA.. uyir",
    "Itinum dislike adikkan aalundallo...nallatine angeegarikkan padikk... duranthangal..."
]

predictions = predict(texts, model, tokenizer, device)
print("Predictions:", predictions.numpy())



Predictions: [0 1]
